### (0) IMPORT

In [11]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
from selenium import webdriver
from sklearn.cluster import KMeans
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import spotipy
import pandas as pd

#BORROWING KAISHENGS CLIENT SECRET AND ID THANKS KS BB PLS DONT DISABLE YA
auth_manager = SpotifyClientCredentials(client_id="21ff73a9b5a94ea8b3a969b906baead1", client_secret="3761e7947ef542149467196a07cf2563")
sp = spotipy.Spotify(auth_manager=auth_manager)

### (1) Playlist extractor - Make sure you do not already have a file called "User_Playlist.csv", else it will be replaced.

In [18]:
import time

# def extract_playlistID(playlists,find_playlist):
#     while True:
#         for playlist in playlists['items']:
#         ##Loop and return playlistID if found
#             if playlist["name"] == find_playlist:
#                 return playlist['id']
#         ##Show next 50 songs
#         if playlists['next']:
#             playlists = sp.next(playlists)
#         else:
#             return None
        
def extract_trackID(playlist_tracks):
    trackID_list = []
    for tracks in playlist_tracks["items"]:
        track_ID = tracks["track"]["id"]
        trackID_list.append(track_ID)
    return trackID_list

def extract_nexttrackID(playlist_tracks):
    trackID_list = []
    for tracks in playlist_tracks["items"]:
        track_ID = tracks["track"]["id"]
        trackID_list.append(track_ID)
    return trackID_list

def extract_songInfo(song):
    info = sp.track(song)
    ##Define features u wanna extract here
    id = song
    songName = info["name"]
    albumName = info["album"]["name"]
    albumID = info["album"]["id"]
    artists = info["artists"][0]["name"]
    for artist in info["artists"]:
        artist_genre = sp.artist(artist["id"])
        genre = artist_genre["genres"]
    explicit = info["explicit"]
    releaseDate = info["album"]["release_date"]
    return [id,songName,albumName,albumID,artists,genre,explicit,releaseDate]

def extract_features(song):
    info = sp.audio_features(song)[0]
    ##Define features u wanna extract here
    danceability = info["danceability"]
    energy = info["energy"]
    key = info["key"]
    loudness = info["loudness"]
    mode = info["mode"]
    speechiness = info["speechiness"]
    acousticness = info["acousticness"]
    instrumental = info["instrumentalness"]
    liveness = info["liveness"]
    valence = info["valence"]
    tempo = info["tempo"]
    duration_ms = info["duration_ms"]
    time_signature = info["time_signature"]
    return [danceability,energy,key,loudness,mode,speechiness,acousticness,instrumental,liveness,
            valence,tempo,duration_ms,time_signature]

#Auth Stuff
auth_manager = SpotifyClientCredentials(client_id="21ff73a9b5a94ea8b3a969b906baead1", client_secret="3761e7947ef542149467196a07cf2563")
sp = spotipy.Spotify(auth_manager=auth_manager)

##INDICATE THE PLAYLIST U WANNA ADD FROM THE USER (IN THIS CASE SPOTIFY IS THE USER AND TOP 50 IS THE PLAYLIST)
# user = "spotify"
# playlists = sp.user_playlists(user, limit=50)

## Hide this if you are inserting your own playlist ID
# for find_playlist in playlists_name:
#     found_ID = extract_playlistID(playlists, find_playlist)
#     if found_ID != None:
#         playlists_ID.append(found_ID)
    
playlists_name = []
playlists_ID = []

## Ask to insert playlist ID
while True:
    url = input("Insert Spotify Playlist ID (Else type 'done'): ")
    if url == "done":
        break
    else:
        playlists_ID.append(url)
    
# Get Playlist Name
print("Obtaining the Following Playlists: ")
for i in playlists_ID:
    playlist_info = sp.playlist(i)
    name = playlist_info["name"]
    print(name)
    playlists_name.append(name)

##Get Info in Playlist with PlaylistID 
tracksID = {}
for i in range(len(playlists_ID)):
    track_list = []
    playlist_tracks = sp.playlist(playlists_ID[i])
    playlist_tracks = playlist_tracks["tracks"]
    track_list = extract_trackID(playlist_tracks)
    for page in range(0,2):
        if playlist_tracks['next']:
            print("More than 100 Songs Detected. WAITT")
            time.sleep(30)
            print("Looking at next 100 songs")
            playlist_tracks = sp.next(playlist_tracks)
            track_list += extract_nexttrackID(playlist_tracks)
    tracksID[playlists_name[i]] = track_list


##Get Other Track Details
results_tocsv = []
results_tocsv.append(["id","songName","albumName","albumID","artist","genre","explicit","releaseDate",
                     "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
                    "valence","tempo","duration_ms","time_signature"])
for i in playlists_name:  
    count = 0
    for song in tracksID[i]:
        time.sleep(1)
        song_info = extract_songInfo(song)
        ##Get Audio Features
        song_features = extract_features(song)
        song_info.extend(song_features)
        results_tocsv.append(song_info)
        count += 1
        
        if count == 100:
            print("Trying not to let spotify kill me. 30s wait :D")
            time.sleep(30)
            count = 0

##Use pandas to convert to dataframe
df = pd.DataFrame(results_tocsv[1:], columns = results_tocsv[0])

##Export to CSV
file_name = "User_Playlist.csv"
df.to_csv(file_name, encoding='utf_8_sig')
print("20s Cooldown...")
time.sleep(20)

KeyboardInterrupt: Interrupted by user

### (2) FUNCTIONSSS

In [3]:
def ohe_prep(df, column, new_name): 
    ''' 
    Create One Hot Encoded features of a specific column
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used
        
    Output: 
    tf_df: One-hot encoded features 
    '''
    
    tf_df = pd.get_dummies(df[column])
    
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)  
    return tf_df

def get_features_database(databaseDF, feature_weights):
    #Select Features
    databaseDF = databaseDF[["id","songName",
                "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
                "valence","tempo","type"]]

    #OHE Features
    key_ohe = ohe_prep(databaseDF, 'key','key') * feature_weights['key']
    mode_ohe = ohe_prep(databaseDF, 'mode','mode') * feature_weights['mode']

    ##Normalise/Scale Audio Columns
    float_cols = databaseDF.dtypes[databaseDF.dtypes == 'float64'].index.values
    floats = databaseDF[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns)

    # Apply weight to each float column
    for col in floats_scaled.columns:
        floats_scaled[col] *= feature_weights.get(col, 1.0)

    ##Combine all Features
    final = pd.concat([floats_scaled, key_ohe, mode_ohe, databaseDF["type"]], axis = 1)
    return final


feature_weight = {
    "danceability": 0.6,
    "energy": 0.7,
    "key": 0.1,
    "loudness": 0.8,
    "mode": 0.1,
    "speechiness": 0.5,
    "acousticness": 0.3,
    "instrumentalness": 0.5,
    "liveness": 0.3,
    "valence": 0.7,
    "tempo": 0.6,
    "type": 0.1
}

# feature_weight = {
#     "danceability": 1,
#     "energy": 1,
#     "key": 1,
#     "loudness": 1,
#     "mode": 1,
#     "speechiness": 1,
#     "acousticness": 1,
#     "instrumentalness": 1,
#     "liveness": 1,
#     "valence": 1,
#     "tempo": 1,
#     "type": 1
# }

def generate_rec(databaseDF, database_vector, user_vector, genre):
    #Cosine Similarity
    databaseDF["sim"] = cosine_similarity(database_vector,user_vector)

    #Drop rows with different genre from top 3 genres
    databaseDF = databaseDF[(databaseDF["genre"] == genre)]

    rec_top5 = databaseDF.sort_values('sim',ascending = False).head()
    return rec_top5

    #KEEP FOR FUTURE USE IF NEEDED
    # final_reco = []
    
    # # # Drop rows with different genre from top 3 genres
    # # rec_top5 = databaseDF[(databaseDF["genre"] == genre_top3[0]) | (databaseDF["genre"] == genre_top3[1]) | (databaseDF["genre"] == genre_top3[2])]

    # #Sort and recommend top 5 with same genres
    # pre_rec_ordered = databaseDF.sort_values('sim',ascending = False).head(20)

    # #Find genre for each db song in order, and append first 5 songs once genre match is found
    # for i in range(20):
    #     track_data = sp.track(pre_rec_ordered["id"].iloc[i])

    #     artist_ids = []
    #     for artist in track_data["artists"]:
    #         artist_ids.append(artist["id"])

    #         artists_data = sp.artists(artist_ids)

    #     genres = []
    #     for artist in artists_data["artists"]:
    #         ls_art_genre = artist["genres"]

    #         for j in ls_art_genre: #['funk']
    #             genres.append(j)

    #     genres = set(genres) # removes duplicates

    #     #UNCOMMENT LINE BELOW to show top songs even if none of them share same genre (WHEN THERE ARE NO SONGS RECOMMENDED)
    #     # print([pre_rec_ordered['artist'].iloc[i], pre_rec_ordered['songName'].iloc[i], pre_rec_ordered['sim'].iloc[i], genres]) 

    #     for g in genres:
    #         if g in genre_top3:
    #             final_reco.append([pre_rec_ordered['artist'].iloc[i], pre_rec_ordered['songName'].iloc[i], pre_rec_ordered['sim'].iloc[i], genres])
                
    # return final_reco
    



### (3) NORMALISING Genre Names
If METHOD1 does not work for particular song, do METHOD2 so on and so forth <br><br>
METHOD1 - Find User Songs in Dataset and Copy Over <br>
METHOD2 - Webscrap from get_genre.com which genre name is generic and similar to dataset<br>
METHOD3 - SPOTIFYAPI


<hr/><br>
METHOD2 - Webscraping Function: REQUIRES SELENIUM AND BS4 LIBRARY. FOR SELENIUM NEEDS TO INSTALL CHROMEDRIVERS TOO from below: <br>


[CHROMEDRIVERS](https://googlechromelabs.github.io/chrome-for-testing) 

In [ ]:
# # !pip install bs4
# !pip install selenium==3.141.0 


In [12]:


##GENRE ENGINEERING
genre_dict = {
    "pop": ["pop punk", "pop rap", "pop rock", "synthpop","dance-pop", "singer-songwriter"],
    "rock": ["psych rock","power pop","alt rock", "hard rock","emo","blues", "folk", "garage", "pop rock", "acoustic", "rock and roll", "singer-songwriter", ""],
    "metal": ["metalcore","black metal", "death metal", "heavy metal", "goth", "gothic metal", "groove", "punk", "grindcore", "industrial", "alternative metal"],
    "house": ["chicago house", "progressive house", "deep house", "edm", "electro house", "future house", "tech house"],
    "country": ["country pop", "country blues", "country rap", "country rock"],
    "r&b": ["soul","dance","contemporary r&b", "alternative r&b", "soul jazz", "r&b and soul","rhythm and blues"],
    "techno": ["detroit techno","minimal techno", "hardcore"],
    "electro": ["drum and bass","dubstep","electronic", "club", "electro house", "electronic dance music", "electro swing", "electropop"],
    "hip hop": ["rap", "funk", "alternative hip hop", "rap rock", "rap metal", "jazz rap"],
    "k-pop": [],
    "indie": ["indie rock", "indie pop"]
}

def generaliseGenre(genre):
    for key,value in genre_dict.items():
        ##if genre scrapped is same as generic genre(key), return genre
        if key == genre:
            return key
        else:
            ##iterate through all the genres (value) in the dict to find suitable generic genre
            for alt_genre in value:
                if alt_genre == genre:
                    return key
    ##return none if no genre found
    return None

def scrap_genre(artistName):
    ##URL of website to scrap
    url = f"https://www.getgenre.com/artist/{artistName}"


    ##CHANGE PATH TO PATH OF DOWNLOADED CHROMEDRIVER
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(options=options)
    
    # WAIT FOR BUTTON TAG TO LOAD BEFORE CONTINUING
    driver.implicitly_wait(20)
    driver.get(url)
    
    try:
        classtext = "MuiButtonBase-root MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeMedium MuiButton-outlinedSizeMedium MuiButton-root MuiButton-outlined MuiButton-outlinedPrimary MuiButton-sizeMedium MuiButton-outlinedSizeMedium css-x3ahaf"
        button = driver.find_element(By.XPATH, '//*[@id="genres-text"]/div/button')
        time.sleep(5)
        # Parse the HTML content of the button with BS4
        html = button.get_attribute("outerHTML")
        driver.close()
        driver.quit()

        #Use BS4 to read HTML segement and extract Genre Text
        soup = BeautifulSoup(html, "html.parser")
        # Find html button tag with class to obtained genre text
        genre = soup.find('button', class_=classtext).get_text().lower()
        result = generaliseGenre(genre)
        return result
    except:
        driver.close()
        driver.quit()
        pass


In [13]:
##Import User Playlist
userDF = pd.read_csv("User_Playlist.csv")
userDF = userDF.drop(columns=["Unnamed: 0"])
##Import Dataset
df = pd.read_csv("spotify_dataset_eda.csv")
df = df.drop(columns=["Unnamed: 0"])
df = df.dropna()

#Extract all genres from dataset
datasetGenres_list = []
for i in df["genre"]:
    if i not in datasetGenres_list:
        datasetGenres_list.append(i)

song_genres_list = []

## Dict for scrapped genres to avoid scraping same artist multiple times
scrap_dict = {}

##Find Playlist Songs in Database and replace genre
for index, row in userDF.iterrows():
    id = row["id"]
    artistName = row["artist"]

    ##List to include all genres from one song
    one_song_genre = []

    ##Check if artistName is already scrapped or tracked
    if artistName in scrap_dict.keys():
        genre = scrap_dict[artistName]
        one_song_genre.append(genre)
    else:
        ##variable to check if song in database
        dataset_genre = df[df["id"] == id]

        if not dataset_genre.empty:
            ##METHOD1 - Searches for kaggle ds for song inputted to retrieve genre (matching kaggle ds). 
        ##If found, genre from kaggle ds replaces highly specific genre of song retrieved from spotify
            one_song_genre = dataset_genre["genre"].to_list()
            for i in one_song_genre:
                scrap_dict[artistName] = i
        else:
            ##METHOD2 - Web Scraping to retrieve simplified genre from getgenre.com to replace highly specific genre retrieved from spotify
            genre = scrap_genre(artistName)
            scrap_dict[artistName] = genre
            if genre != "":
                one_song_genre.append(genre)

    ##CHECKS IF METHOD 1 or 2 managed to obtain generalised genre data
    song_genres_list.append(one_song_genre)

userDF["genre"] = [genre[0] for genre in song_genres_list]
userDF = userDF.dropna(subset=["genre"])
userDF

,id,songName,albumName,albumID,artist,genre,explicit,releaseDate,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,duration_ms,time_signature
0,37BH3RrKhRaHMQwuhlkEaF,Bummed,Bummed,38AOH6bBpJDYTn1EAaCMSn,Chet Porter,electro,True,2020-03-05,0.437,0.759,...,-6.133,1,0.039,0.000371,0,0.0872,0.382,160.034,184800,4


### (4) CLUSTERING GANG GANG

In [14]:
import pandas as pd
from sklearn.cluster import KMeans

# Assuming userDF is an existing dataframe
df = userDF.copy()

# features for clustering
X = df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumental', 'liveness', 'valence', 'tempo']]


#choose how many clusters u want
if df.shape[0] < 4:
    number_clusters = df.shape[0]
else:
    number_clusters = 3


# Perform KMeans clustering
kmeans = KMeans(n_clusters=number_clusters, random_state=0)
df['cluster'] = kmeans.fit_predict(X)

df.sort_values(by="cluster")

#Create new feature/column of artist_songName to remove duplicate later on
df["artist_songName"] = df["artist"] + "_" + df["songName"]

#Check for duplicates in user Playlist
df = df.drop_duplicates(subset=["artist_songName"],ignore_index= True)


/Users/kaisheng/miniconda3/envs/Spotify/lib/python3.12/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


### (4) Find Top 3 Genres in User Playlist By Counting Frequency of Genres of each cluster

In [15]:
clusters = {}
topgenre_clusters = []
allgenres = []

# Output each cluster to a separate dataframe
for cluster_num in range(number_clusters):
    cluster_df = df[df['cluster'] == cluster_num].copy()  # Make a copy to avoid modifying the original dataframe
    clusters[cluster_num] = cluster_df

    common_genre = cluster_df['genre'].mode()[0]  # Find the most common genre in the cluster

    #newlist for top genres in each cluster
    topgenre_clusters.append(common_genre)
    allgenres = df['genre'].unique().tolist()

print("Top 3 Genre's of User Playlist:" + str(topgenre_clusters))

Top 3 Genre's of User Playlist:['electro']


### (5) Extract Both User and Dataset Csv

In [16]:
#Extract Database CSV
databaseDF = pd.read_csv("spotify_dataset_eda.csv", encoding="utf_8_sig")
databaseDF = databaseDF.drop(columns=['Unnamed: 0'])
#Selects columns that we want
databaseDF = databaseDF[["id","artist","songName", "genre","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
        "valence","tempo",]]

#Create new feature/column of artist_songName
databaseDF["artist_songName"] = databaseDF["artist"] + "_" + databaseDF["songName"]

##Change all values to lowercase
databaseDF["artist_songName"] = databaseDF["artist_songName"].str.lower()

#Check for duplicates in database
databaseDF = databaseDF.drop_duplicates(subset=["artist_songName"],ignore_index= True)

ITERATE THROUGH ALL THE CLUSTERS FUCKU JAE

In [17]:

result = pd.DataFrame()  # Initialize finalrec as an empty DataFrame


for i in range(len(topgenre_clusters)):
    clusternumber = i
    genre = topgenre_clusters[i]
    #pullout existing user playlist clusters
    playlistDF = pd.DataFrame(clusters[clusternumber])

    #Selects columns that we want
    playlistDF = playlistDF[["id","artist","songName", "genre","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
            "valence","tempo",]]


    # Get the column names of each DataFrame
    playlist_columns = set(playlistDF.columns)
    database_columns = set(databaseDF.columns)

    # Find columns in playlistDF that are not in databaseDF
    columns_in_playlist_not_in_database = playlist_columns - database_columns

    # Find columns in databaseDF that are not in playlistDF
    columns_in_database_not_in_playlist = database_columns - playlist_columns



    ## Merge user + dataset dataframe to normalise 
    #Normalise takes min and max in dataframe as reference and change it to 0 and 1 respectively
    #Group the dataframe as we gonna split it again alter
    databaseDF["type"] = "Dataset"
    playlistDF["type"] = "User"

    #Merge the 2 datasets together
    combinedDF = pd.concat([databaseDF,playlistDF], ignore_index=True)

    #Check for duplicates between user and Database
    combinedDF = combinedDF.drop_duplicates(subset=["artist_songName"], keep="last",ignore_index= True)

    ##Update databaseDF with removed songs from user Playlist
    databaseDF = combinedDF[(combinedDF["type"]== "Dataset")]

    ##Normalise and get Vectors for Dataset + User
    normalised_vector = get_features_database(combinedDF, feature_weight)

    ##Seperate User from databaseDF 
    database_vector = normalised_vector[normalised_vector["type"] == "Dataset"]
    user_vector = normalised_vector[normalised_vector["type"] == "User"]

    #Drop "type" column
    database_vector = database_vector.drop(columns="type")
    user_vector = user_vector.drop(columns="type")
    databaseDF = databaseDF.drop(columns=["type"])



    ##Single Vector Creation
    final_user_vector_list = []
    for i in user_vector.columns:
        final_user_vector_list.append(user_vector[i].sum()/len(user_vector[i]))

    #Putting into a vector dataframe
    final_user_vector = pd.DataFrame(columns=user_vector.columns,)
    final_user_vector.loc[0] = final_user_vector_list

    final_user_vector


    ##Recommendation Time!
    def generate_rec(databaseDF, database_vector, user_vector, genre, finalrec):
        # Cosine Similarity
        databaseDF["sim"] = cosine_similarity(database_vector, user_vector)

        # Drop rows with different genre from top 3 genres
        databaseDF = databaseDF[databaseDF["genre"].astype(str) == str(genre)]

        rec_top5 = databaseDF.sort_values('sim', ascending=False).head()
        num_rows = rec_top5.shape[0]

        rec_top5['cluster'] = clusternumber  # Assuming clusternumber is defined elsewhere
        df_concatenated = pd.concat([finalrec, rec_top5], ignore_index=True)
        return df_concatenated  # Return the concatenated DataFrame

    ##Get genre of particular cluster and run recommendation function
    genre = topgenre_clusters[clusternumber]
    ##Combine recommended songs from all 3 clusters
    final = generate_rec(databaseDF, database_vector, final_user_vector, genre, result) 
    result = final
    

result


,id,artist,songName,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,artist_songName,sim,cluster
0,5miFbeLmMdrSlrrPTMWHua,Kasger,Highland,electro,0.446,0.827,8,-6.893,1,0.0354,0.000446,0.017100,0.0877,0.311,174.070,kasger_highland,0.997071,0
1,5PAO1KWruUA0lSCJwBjFBu,Maduk,The End,electro,0.456,0.939,8,-4.402,1,0.0487,0.000603,0.000008,0.0463,0.404,173.999,maduk_the end,0.996918,0
2,6ijcMlRm3iTX2Cjvf19RLQ,Maduk,Not Alone - Radio Edit,electro,0.469,0.952,8,-3.533,1,0.0431,0.001960,0.017700,0.0799,0.401,173.938,maduk_not alone - radio edit,0.996896,0
3,2fJzvVoZkY4iuwK1EUwbvr,Kavinsky,Zombie,electro,0.535,0.717,8,-7.601,1,0.0348,0.000392,0.001120,0.0501,0.398,144.980,kavinsky_zombie,0.996876,0
4,3agPUfkVZ5B2If4vXAyHaj,Shiny Toy Guns,Money For That,electro,0.335,0.755,8,-4.290,1,0.0353,0.005400,0.000000,0.1250,0.329,157.950,shiny toy guns_money for that,0.996871,0
